In [16]:
def stock_data():
    
    """
    日本証券取引所から上場企業リストを取得する
    ETFを除外した銘柄一覧を整形し、"stocklist.csv"として保存する
    """
    import pandas as pd
    import requests
    import pandas as pd
    
    url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    r = requests.get(url)
    with open('data_j.xls', 'wb') as output:
        output.write(r.content)
        
    stocklist = pd.read_excel("./data_j.xls")
    
    stocklist = stocklist[(stocklist["市場・商品区分"] == "市場第一部（内国株）")|
                                  (stocklist["市場・商品区分"] == "JASDAQ(スタンダード・内国株）") |
                                  (stocklist["市場・商品区分"] == "JASDAQ(グロース・内国株）") |
                                  (stocklist["市場・商品区分"] == "マザーズ（内国株）") |
                                  (stocklist["市場・商品区分"] == "市場第二部（内国株）")]
    
    stocklist = stocklist[["コード","銘柄名","市場・商品区分","33業種区分","規模区分"]]
    
    stocklist = stocklist.replace({'市場第一部（内国株）': '東証1部',
                   "市場第二部（内国株）": "東証2部",
                   'JASDAQ(スタンダード・内国株）': "JASDAQ",
                   'JASDAQ(グロース・内国株）': "JASDAQ",
                   'マザーズ（内国株）': "マザーズ"})  
    
    stocklist.reset_index(drop=True)
    stocklist.to_csv("stocklist.csv", index = False)

In [14]:
def conpany_url():
    
    """
    stock_data()関数から取得したstocklist.csvからコードを取得する
    取得したコードをもとに日経新聞社から企業URLをスクレイピングする
    取得した企業URLを"url_list.csv"として保存する
    """
    
    import pandas as pd
    import time
    from tqdm import tqdm
    import urllib.request
    from bs4 import BeautifulSoup
    import csv

    stocklist = pd.read_ｃｓｖ("./stocklist.csv")
    code_list = stocklist["コード"]

    url_list = []
    error_list = []

    for code in tqdm(code_list):
        try:
            url = "https://www.nikkei.com/nkd/company/gaiyo/?scode=" + str(code)
            html = urllib.request.urlopen(url)
            soup = BeautifulSoup(html, "html.parser")
            code_soup = soup.find_all(class_="w668 a-tblFixed")
            code_soup = code_soup[0]
            url_tag =  code_soup.find_all(class_="a-w80p")[1]
            url = url_tag.find('a')
            url = url.get('href')
            url_list.append(url)
            time.sleep(1)
        except Exception as e:
            print(e)
            print(type(e))
            print(code)
            error_list.append(code)
            url_list.append("Nan")
            pass
    print("URLリスト:", url_list)
    print("エラーリスト:", error_list)
    f = open('url_list.csv', 'w')
    for x in url_list:
        f.write(str(x) + "\n")
    f.close()